In [2]:
import os
%pwd

'/Users/shreyashsahare28/Documents/SHREYASH/MASTERS/ADDITIONAL_PROJECTS/shres-mlops-project/research'

In [3]:
os.chdir('../')

In [4]:
%pwd

'/Users/shreyashsahare28/Documents/SHREYASH/MASTERS/ADDITIONAL_PROJECTS/shres-mlops-project'

In [5]:
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class DataIngestionConfig:
    root_dir: Path
    source_URL: str
    local_data_file: Path
    unzip_dir: Path

In [6]:
from mlops_proj.constants import *
from mlops_proj.utils.common import read_yaml, create_directories

In [7]:
class ConfigurationManager:
    def __init__(
            self,
            config_filepath = CONFIG_FILE_PATH,
            params_filepath = PARAMS_FILE_PATH,
            schema_filepath = SCHEMA_FILE_PATH):
        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        self.schema = read_yaml(schema_filepath)

        create_directories([self.config.artifacts_root])
    
    def get_data_ingestion_config(self) -> DataIngestionConfig:
        config = self.config.data_ingestion

        create_directories([config.root_dir])

        data_ingestion_config = DataIngestionConfig(
            root_dir=config.root_dir,
            source_URL=config.source_URL,
            local_data_file=config.local_data_file,
            unzip_dir=config.unzip_dir
        )

        return data_ingestion_config

In [9]:
import os
from mlops_proj import logger
from urllib.request import urlretrieve
import zipfile
from mlops_proj.utils.common import get_size

In [10]:
class DataIngestion:
    def __init__(self, config = DataIngestionConfig):
        self.config = config
    

    def download_file(self):
        if not os.path.exists(self.config.local_data_file):
            filename, headers = urlretrieve(
                url=self.config.source_URL,
                filename=self.config.local_data_file
            )
            logger.info(f"{filename} downloaded! Here's info about the file: \n {headers}")
        else:
            logger.info(f"File already exists of size: {get_size(Path(self.config.local_data_file))}")
    

    def extract_zipfile(self):
        unzip_path = self.config.unzip_dir
        os.makedirs(unzip_path, exist_ok=True)
        with zipfile.ZipFile(self.config.local_data_file, 'r') as zip_ref:
            zip_ref.extractall(unzip_path)

In [13]:
try:
    # initializing and creating the configuration object
    config = ConfigurationManager()

    # getting the data ingestion configuration from the "config.yaml" file
    data_ingestion_config = config.get_data_ingestion_config()

    # creating the data ingestion object
    data_ingestion = DataIngestion(config = data_ingestion_config)

    # downloading the dataset file using the object
    data_ingestion.download_file()

    # extracting the downloaded file at the target unzipping directory which was already mentioned in the configuration file
    data_ingestion.extract_zipfile()
except Exception as e:
    raise e

[2025-07-11 20:46:44,734]: INFO: common: yaml file: config/config.yaml loaded successfully
[2025-07-11 20:46:44,735]: INFO: common: yaml file: params.yaml loaded successfully
[2025-07-11 20:46:44,735]: INFO: common: yaml file: schema.yaml loaded successfully
[2025-07-11 20:46:44,736]: INFO: common: created directory at: artifacts
[2025-07-11 20:46:44,737]: INFO: common: created directory at: artifacts/data_ingestion
[2025-07-11 20:46:45,376]: INFO: 614481981: artifacts/data_ingestion/data.zip downloaded! Here's info about the file: 
 Connection: close
Content-Length: 23329
Cache-Control: max-age=300
Content-Security-Policy: default-src 'none'; style-src 'unsafe-inline'; sandbox
Content-Type: application/zip
ETag: "c69888a4ae59bc5a893392785a938ccd4937981c06ba8a9d6a21aa52b4ab5b6e"
Strict-Transport-Security: max-age=31536000
X-Content-Type-Options: nosniff
X-Frame-Options: deny
X-XSS-Protection: 1; mode=block
X-GitHub-Request-Id: 7F36:341ACF:322028:41D8A4:6871CC92
Accept-Ranges: bytes
Dat

**Note:** If the `.yaml` files are empty, the data ingestion pipeline won't work since 